In [70]:
import requests
import pandas as pd
import io
from tqdm import tqdm
import numpy as np

In [54]:
BASE_URL = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main"

In [55]:
def fetch_documents():
    print("Fetching documents...")
    relative_url = "04-monitoring/data/results-gpt4o-mini.csv"
    docs_url = f"{BASE_URL}/{relative_url}?raw=1"
    docs_response = requests.get(docs_url)
    
    if docs_response.status_code == 200:
        # Read the CSV content
        csv_content = docs_response.content.decode('utf-8')
        # Load CSV content into a pandas DataFrame
        df = pd.read_csv(io.StringIO(csv_content))
        print(f"Fetched {len(df)} documents")
        return df
    else:
        print(f"Failed to fetch documents. Status code: {docs_response.status_code}")
        return pd.DataFrame()

In [56]:
results_gpt4o = fetch_documents()
print(results_gpt4o.head())
print("Rows in df: ", len(results_gpt4o))

Fetching documents...
Fetched 1830 documents
                                          answer_llm  \
0  You can sign up for the course by visiting the...   
1  You can sign up using the link provided in the...   
2  Yes, there is an FAQ for the Machine Learning ...   
3  The context does not provide any specific info...   
4  To structure your questions and answers for th...   

                                         answer_orig  document  \
0  Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
1  Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
2  Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
3  Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   
4  Machine Learning Zoomcamp FAQ\nThe purpose of ...  0227b872   

                                            question  \
0                Where can I sign up for the course?   
1                 Can you provide a link to sign up?   
2  Is there an FAQ for this Machine Learning course? 

In [57]:
#limiting df to 300 first 300 entities

results_gpt4o = results_gpt4o.iloc[:300]
print("Rows in df: ", len(documents_df))

Rows in df:  300


In [58]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [59]:
# Function to encode a query using the embedding model
def get_query_vector(query):
    return embedding_model.encode(query)

In [60]:
# Encode the specific sentence and print the first vector value
answer_llm = results_gpt4o.iloc[0].answer_llm
query_embedding = get_query_vector(answer_llm)
print("First vector value of the encoded user query:", query_embedding[0])

First vector value of the encoded user query: -0.42244655


In [61]:
results_gpt4o_oriented = results_gpt4o.to_dict(orient='records')

record = results_gpt4o_oriented[0]

record

{'answer_llm': 'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).',
 'answer_orig': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'document': '0227b872',
 'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp'}

In [64]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = get_query_vector(answer_llm)
    v_orig = get_query_vector(answer_orig)
    
    return v_llm.dot(v_orig)

In [66]:
similarity = []

for record in results_gpt4o_oriented:
    sim = compute_similarity(record)
    similarity.append(sim)

In [68]:
results_gpt4o['cosine'] = similarity
results_gpt4o['cosine'].describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547924
25%       24.307844
50%       28.336870
75%       31.674309
max       39.476013
Name: cosine, dtype: float64

In [74]:
#making normzlization function
def norm(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

#normalizing cozine similarity vecrors
def compute_similarity_norm(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = norm(get_query_vector(answer_llm))
    v_orig = norm(get_query_vector(answer_orig))

    return v_llm.dot(v_orig)

In [75]:
similarity_norm = []

for record in results_gpt4o_oriented:
    sim_norm= compute_similarity_norm(record)
    similarity_norm.append(sim_norm)

In [76]:
results_gpt4o['cosine_norm'] = similarity_norm
results_gpt4o['cosine_norm'].describe()

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: cosine_norm, dtype: float64

In [87]:
from rouge import Rouge
rouge_scorer = Rouge()

def rouge_score(r):
    scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
    return scores

In [95]:
# results_gpt4o.loc[results_gpt4o['document'] == '5170565b']

item = results_gpt4o.iloc[10]

rouge_full = rouge_score(item)

rouge_full

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [105]:
def avg_rouge_score(r):
    # Extract ROUGE scores
    rouge_1 = r['rouge-1']
    rouge_2 = r['rouge-2']
    rouge_l = r['rouge-l']
    
    # Compute average for recall, precision, and F1-score
    avg_r = np.mean([rouge_1['r'], rouge_2['r'], rouge_l['r']])
    avg_p = np.mean([rouge_1['p'], rouge_2['p'], rouge_l['p']])
    avg_f = np.mean([rouge_1['f'], rouge_2['f'], rouge_l['f']])
    
    return {'avg_r': avg_r, 'avg_p': avg_p, 'avg_f': avg_f}


In [106]:
avg_rouge_score(rouge_full)

{'avg_r': 0.35490035490035493,
 'avg_p': 0.35490035490035493,
 'avg_f': 0.35490034990035496}

In [116]:
def extract_rouge_f(r):
    rouge_1 = r['rouge-1']['f']
    rouge_2 = r['rouge-2']['f']
    rouge_l = r['rouge-l']['f']
    return rouge_1, rouge_2, rouge_l

In [118]:
rouge_1_scores = []
rouge_2_scores = []
rouge_l_scores = []

# Compute ROUGE scores for each record and extract the f-scores
for record in results_gpt4o_oriented:
    rouge_scores = rouge_score(record)
    rouge_1, rouge_2, rouge_l = extract_rouge_f(rouge_scores)
    rouge_1_scores.append(rouge_1)
    rouge_2_scores.append(rouge_2)
    rouge_l_scores.append(rouge_l)


In [123]:
len(rouge_1_scores)

rouge_1_scores

[0.09523809178130524,
 0.12499999641113292,
 0.41558441095631643,
 0.2162162117421476,
 0.14207649881095297,
 0.43137254522106894,
 0.41269840791131274,
 0.30434782321361065,
 0.5172413747919143,
 0.34374999517578125,
 0.45454544954545456,
 0.6060606010606061,
 0.753623183415249,
 0.7297297247881666,
 0.6849315018952901,
 0.4772727222727273,
 0.47999999515022235,
 0.3888888841358025,
 0.2857142807256236,
 0.37499999505,
 0.24719100710011369,
 0.2682926793664485,
 0.5242718399095109,
 0.18918918665814466,
 0.5048543641813555,
 0.6206896501724138,
 0.3333333290589569,
 0.4130434736011342,
 0.633663361446917,
 0.359550557257922,
 0.4489795870054144,
 0.5217391255198489,
 0.4090909041322315,
 0.25714285306122453,
 0.16666666172839517,
 0.19999999601250007,
 0.29999999601250005,
 0.42857142429705225,
 0.23255813514332083,
 0.7096774143756504,
 0.4848484798530762,
 0.3934426180166622,
 0.5245901590002687,
 0.3692307642414202,
 0.22222221852839513,
 0.11111110797839514,
 0.15789473383656513,


In [124]:
results_gpt4o['rouge_1'] = rouge_1_scores
results_gpt4o['rouge_2'] = rouge_2_scores
results_gpt4o['rouge_l'] = rouge_l_scores

results_gpt4o['rouge_2'].describe()

results_gpt4o['rouge_2'].mean()

0.20696501983423318